In [ ]:
%load_ext autoreload
%autoreload 2
import copy
import gc
import sys
import pickle

sys.path.append('..')

import numpy as np
import pandas as pd
import scipy.stats as sps

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm

from sklearn.model_selection import (GridSearchCV, RandomizedSearchCV, 
                                     cross_val_score)
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

import xgboost as xgb
from xgboost import XGBRegressor

from hyperopt import hp, tpe, STATUS_OK, Trials
from hyperopt.fmin import fmin

from src.utils.cross_validation import TimeSeriesGroupSplit
from src.utils.downcasting import downcast_dtypes

sns.set(font_scale=1.2)
%matplotlib inline

In [ ]:
max_text_features = 20
random_state = 42

MEAN_CONSTANT = 0.3343

# XGBoost

In this notebook we will produce predictions by XGBoost.

## Preparing datasets

In this section we will load all datasets and prepare them for training.

In [ ]:
sales_train = pd.read_csv('../data/processed/sales_train.csv')
train = pd.read_feather('../data/processed/train.ftr')
test = pd.read_feather('../data/processed/test.ftr')

items = pd.read_csv('../data/processed/items.csv')
tfidf_truncated_svd = pd.read_feather('../data/processed/text/tfidf_truncated-svd.ftr')

In [ ]:
train.drop(columns=['index'], inplace=True)
test.drop(columns=['index', 'level_0'], inplace=True)

### Adding text features

In [ ]:
tfidf_truncated_svd = tfidf_truncated_svd[tfidf_truncated_svd.columns[:max_text_features]]

In [ ]:
tfidf_truncated_svd['item_id'] = items.item_id

In [ ]:
train = pd.merge(
    train,
    tfidf_truncated_svd,
    how='left', on='item_id'
)

test = pd.merge(
    test,
    tfidf_truncated_svd,
    how='left', on='item_id'
)

gc.collect();

### Clipping target

According to evaluation, target will be clipped between 0 and 20. Let's do it in our dataset.

In [ ]:
train.target = np.clip(train.target, 0, 20)

### Process categorical data

Find categorical columns.

In [ ]:
categorical_types = ['object', 'bool']

In [ ]:
train.dtypes[np.isin(train.dtypes.values, categorical_types)]

Don't touch boolean objects, they are already label encoded.

Let's remove columns `item_name`, `shop_name`, because we already have them label encoded as `item_id`, `shop_id`.

In [ ]:
train.drop(columns=['item_name', 'shop_name'], inplace=True)
test.drop(columns=['item_name', 'shop_name'], inplace=True)

Let's define list with all categorical values.

In [ ]:
categorical_features = [
    'month', 'item_id', 'item_full_category_name', 'item_category_name', 
    'item_subcategory_name', 'shop_id', 'city'
]

#### Label encoding

We will label encode only `city`, because in other cases we have already label encoded features or there are values on test, that are not present on train.

In [ ]:
label_encode_features = ['city']

for column in label_encode_features:
    le = LabelEncoder()
    
    encoded_feature_train = le.fit_transform(train[column])
    train[f'{column}_labeled'] = encoded_feature_train
    
    encoded_feature_test = le.transform(test[column])
    test[f'{column}_labeled'] = encoded_feature_test

#### Mean encoding

In [ ]:
for column in tqdm(categorical_features):
    # encode train
    cumsum = train.groupby(column).target.cumsum() - train.target
    cumcount = train.groupby(column).cumcount()
    encoded_feature = cumsum / cumcount
    encoded_feature.fillna(MEAN_CONSTANT, inplace=True)
    
    train[f'{column}_mean_encoded_mean'] = encoded_feature
    
    # encode test
    mean_train = train.groupby(column).target.mean()
    test[f'{column}_mean_encoded_mean'] = test[column].map(mean_train).fillna(MEAN_CONSTANT)

Drop all redundant columns.

In [ ]:
to_drop = ['item_id', 'item_full_category_name', 'item_category_name', 
           'item_subcategory_name', 'city']
train.drop(columns=to_drop, inplace=True)
test.drop(columns=to_drop, inplace=True)

### Processing NaNs

Fill NaNs.

In [ ]:
train.columns[train.isna().sum() > 0]

As we expected there are some problems only with `num_residents`. We can fill it with zero, because it will be border value for this feature, trees can handle it properly.

In [ ]:
train.fillna(0, inplace=True)
test.fillna(0, inplace=True)

### Removing target

Remove target from train.

In [ ]:
y = train.target
train.drop(columns=['target'], inplace=True)

### Creation of validation split

Let's also delete from train rows that appears only on validation, it will make our train/validation split more consistant with train/test split.

In [ ]:
X_valid = train[train.date_block_num == 33]
X_train = train[train.date_block_num < 33]
y_valid = y[train.date_block_num == 33]
y_train = y[train.date_block_num < 33]
X_test = test
    
del train, test
gc.collect()

## Hyperparameters tuning

In this section we will find optimum parameters for a model. Firstly, fix the result before any optimization.

In [ ]:
ts = TimeSeriesGroupSplit(n_splits=5, max_train_size=int(1.5*10**6))

xgb_params = {
    'objective': 'reg:squarederror',
    'learning_rate': 0.1,
    'seed': random_state
}

In [ ]:
default_score = cross_val_score(
    XGBRegressor(**xgb_params), 
    X_train, y_train, groups=X_train.date_block_num,
    n_jobs=1, 
    scoring='neg_root_mean_squared_error', 
    verbose=0,
    cv=ts
).mean()
print(f'Current score: {-default_score:.5f}')

Score: $0.91020$.

### `n_estimators`

In this section we fix `learning_rate = 0.1` and try to find reasonable num of iterations.

In [ ]:
# X_train_dataset = xgb.DMatrix(X_train[X_train.date_block_num >= 25], 
#                               label=y_train[X_train.date_block_num >= 25])
# X_valid_dataset = xgb.DMatrix(X_valid, label=y_valid)

# xgb.train(xgb_params, X_train_dataset, num_boost_round=500, 
#           evals=[(X_train_dataset, 'train'), (X_valid_dataset, 'test')], 
#           early_stopping_rounds=100)

# del X_train_dataset, X_valid_dataset
# gc.collect();

In [ ]:
# param_grid = {
#     'n_estimators': np.arange(50, 301, 50)
# }

# gs = GridSearchCV(
#     LGBMRegressor(**lgb_params), 
#     param_grid,          
#     n_jobs=1, 
#     scoring='neg_root_mean_squared_error', 
#     verbose=10,
#     refit=False,
#     cv=ts
# )

gs.fit(X_train, y_train, groups=X_train.date_block_num)

gs.cv_results_

gs.best_params_

In [ ]:
xgb_params['n_estimators'] = 243

### Tree parameters, subsampling, regularization

Here we will find optimum values for building tree:
* `max_depth`
* `min_child_weight`

For subsampling:
* `subsample`
* `colsample_bytree`

For regularization:
* `gamma`
* `alpha`
* `lambda`

In [ ]:
xgb_params = {
    'objective': 'reg:squarederror',
    'learning_rate': 0.1,
    'seed': random_state
}

In [ ]:
def objective(params):
    """Function to optimize in hyperopt."""
    params['n_estimators'] = int(params['n_estimators'])
    params['max_depth'] = int(params['max_depth'])
    
    current_params = copy.copy(xgb_params)
    current_params.update(params)    
    score = cross_val_score(
        XGBRegressor(**current_params), 
        X_train, y_train, groups=X_train.date_block_num,
        n_jobs=1, 
        scoring='neg_root_mean_squared_error', 
        verbose=0,
        cv=ts
    ).mean()
    
    print(f'RMSE {-score:.5f} params {params}')
    
    return {
        'loss': -score,
        'status': STATUS_OK,
    }

space = {
    'n_estimators': hp.quniform('n_estimators', 10, 100, 5),
    'max_depth': hp.quniform('max_depth', 3, 12, 1),
    'min_child_weight': hp.loguniform('min_child_weight', 0, np.log(1e3)),
    'subsample': hp.uniform('subsample', 0, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0, 1),
    'gamma': hp.uniform('gamma', 0, 0.5),
    'alpha': hp.choice('alpha', [0, hp.loguniform('alpha_not_zero', np.log(1e-3), 0)]),
    'lambda': hp.choice('lambda', [0, hp.loguniform('lambda_not_zero', np.log(1e-3), 0)])
}

In [ ]:
np.random.seed(random_state)
trials = Trials()
best = fmin(
    fn=objective, 
    space=space, 
    algo=tpe.suggest, 
    max_evals=120,
    trials=trials
)

In [ ]:
with open('../models/xgb/hyperopt/trials.pkl', 'wb') as ouf:
    pickle.dump(trials, ouf)

In [ ]:
best

In [ ]:
xgb_params['max_depth'] = 6
xgb_params['min_child_weight'] = 66.773

xgb_params['subsample'] = 0.576
xgb_params['colsample_bytree'] = 0.898

xgb_params['gamma'] = 0.361
xgb_params['alpha'] = 0
xgb_params['lambda'] = 0

xgb_params['n_estimators'] = 243

### Reduce learning rate

Current score:

In [ ]:
current_score = cross_val_score(
    XGBRegressor(**xgb_params), 
    X_train, y_train, groups=X_train.date_block_num,
    n_jobs=1, 
    scoring='neg_root_mean_squared_error', 
    verbose=0,
    cv=ts
).mean()
print(f'Current score: {-current_score:.5f}')

Score: $0.91447$.

Now we can half our `learning rate` and double `n_estimators`.

In [ ]:
xgb_params

In [ ]:
xgb_params_changed = copy.copy(xgb_params)
xgb_params_changed['learning_rate'] /= 2
xgb_params_changed['n_estimators'] = int(xgb_params['n_estimators'] * 2)

In [ ]:
score_after_change = cross_val_score(
    XGBRegressor(**xgb_params_changed), 
    X_train, y_train, groups=X_train.date_block_num,
    n_jobs=1, 
    scoring='neg_root_mean_squared_error', 
    verbose=0,
    cv=ts
).mean()

print(f'Score after changing: {-score_after_change:.5f}')

Score: $0.91035$.

It helped, but computations was too long.

In [ ]:
xgb_params

## Validation

In this section we will validate best parameters using haldout. We will use not all train, because of limitation of RAM.

In [ ]:
(X_train.date_block_num >= 25).sum()

In [ ]:
start_date_block_num = 25
indices_train = (X_train.date_block_num >= start_date_block_num)
X_train = X_train[indices_train]
y_train = y_train[indices_train]
gc.collect();

In [ ]:
model = XGBRegressor(**xgb_params)
model.fit(X_train, y_train)

In [ ]:
y_predicted = np.clip(model.predict(X_valid), 0, 20)
validation_score = mean_squared_error(y_valid, y_predicted)
print(f'Validation score: {validation_score:.5f}')

Score: $0.81906$

Let's look at predicted values charasteristics.

In [ ]:
pd.Series(y_predicted).describe()

Let's look at feature importances.

In [ ]:
fig, ax = plt.subplots(figsize=(16, 20))
xgb.plot_importance(model, ax=ax);
plt.savefig('../reports/figures/xgb/importances.png', facecolor='white', 
            bbox_inches='tight', pad_inches=0)

In [ ]:
plt.figure(figsize=(16, 20))
image = plt.imread('../reports/figures/xgb/importances.png')
plt.imshow(image, interpolation='spline36')
plt.axis('off')
plt.show()

## Submit

In this section we will train result model and submit prediction. Don't forget to clip values according to [evaluation tab](https://www.kaggle.com/c/competitive-data-science-predict-future-sales/overview/evaluation) (but ay be for tree-based methods it is not necessary).

In [ ]:
X_train = pd.concat((X_train, X_valid))
y_train = pd.concat((y_train, y_valid))
gc.collect();

In [ ]:
X_train.shape

In [ ]:
start_date_block_num = 25
indices_train = (X_train.date_block_num >= start_date_block_num)
X_train = X_train[indices_train]
y_train = y_train[indices_train]
gc.collect();

In [ ]:
model = XGBRegressor(**xgb_params)
bags = 5

bagged_predictions = np.zeros(X_test.shape[0])
for n in tqdm(range(bags)):
    model.set_params(random_state=random_state+n)
    model.fit(X_train, y_train)
    bagged_predictions += np.clip(model.predict(X_test), 0, 20)
    gc.collect()
    
bagged_predictions /= bags

Create submission.

In [ ]:
pd.Series(bagged_predictions).describe()

In [ ]:
submission = pd.read_csv('../data/raw/sample_submission.csv')
submission['item_cnt_month'] = bagged_predictions
submission.to_csv('../models/xgb/submission.csv', index=False)

!kaggle competitions submit competitive-data-science-predict-future-sales -f ../models/xgb/submission.csv -m "XGBoost"

Result is $1.01051$. It is pretty far from top positions.

## OOF predictions

In this section we will create out-of-fold predictions for stacking. We will use cheme f), that was given in the course:
> In time-series task we usually have a fixed period of time we are asked to predict. Like day, week, month or arbitrary period with duration of T.
> 1. Split the train data into chunks of duration T. Select first M chunks.
> 2. Fit N diverse models on those M chunks and predict for the chunk M+1. Then fit those models on first M+1 chunks and predict for chunk M+2 and so on, until you hit the end. After that use all train data to fit models and get predictions for test. Now we will have meta-features for the chunks starting from number M+1 as well as meta-features for the test.
> 3. Now we can use meta-features from first K chunks [M+1,M+2,..,M+K] to fit level 2 models and validate them on chunk M+K+1. Essentially we are back to step 1. with the lesser amount of chunks and meta-features instead of features.

In [ ]:
X_all = pd.concat((X_train, X_test))
test_size = X_test.shape[0]
del X_train, X_test
gc.collect();

In [ ]:
num_blocks = X_all.date_block_num.nunique()
ts = TimeSeriesGroupSplit(n_splits=num_blocks-1, max_train_size=int(1.5*10**6))

We will use chunks, devided by `date_block_num`. In our case $M = 3$, but we won't use all previous chunks to train and limit it according to `max_train_size`.

In [ ]:
model = XGBRegressor(**xgb_params)

predictions = np.zeros(X_all.shape[0])
filled_predictions = np.zeros(X_all.shape[0]).astype(bool)

for i, (train_idx, test_idx) in tqdm(
    enumerate(ts.split(X_all, groups=X_all.date_block_num)), total=22
):
    # skip too small training size
    if i < 2:
        continue
    model.fit(X_all.iloc[train_idx], y_train.iloc[train_idx])
    current_predictions = model.predict(X_all.iloc[test_idx])
    predictions[test_idx] = current_predictions
    filled_predictions[test_idx] = True
    
predictions = predictions[filled_predictions]
y_train = y_train.iloc[filled_predictions[:-test_size]]

Save columns of predictions and clipped predictions.

In [ ]:
OOF_all = pd.DataFrame({'xgb': predictions, 
                        'xgb_clipped': np.clip(predictions, 0, 20)})
OOF_train = OOF_all.iloc[:-test_size].reset_index(drop=True)
OOF_train['target'] = y_train.values
OOF_test = OOF_all.iloc[-test_size:].reset_index(drop=True)

In [ ]:
OOF_train.to_csv('../models/oof/xgb/train.csv', index=False)
OOF_test.to_csv('../models/oof/xgb/test.csv', index=False)